In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
import cv2 as cv

test_img_labels = pd.read_csv("/home/walke/college/cv/ass1/archive/test.csv")
train_img_labels = pd.read_csv("/home/walke/college/cv/ass1/archive/train.csv")

for index in range(test_img_labels.shape[0]):
    img_csv = np.matrix(test_img_labels.iloc[index, 1:].values, dtype=np.uint8).reshape(28, 28)
    edges = cv.Canny(img_csv, 100, 200)
    test_img_labels.iloc[index, 1:] = edges.flatten()

test_img_labels.to_csv("edges_test.csv", index=False)

for index in range(train_img_labels.shape[0]):
    img_csv = np.matrix(train_img_labels.iloc[index, 1:].values, dtype=np.uint8).reshape(28, 28)
    edges = cv.Canny(img_csv, 100, 200)
    train_img_labels.iloc[index, 1:] = edges.flatten()

train_img_labels.to_csv("edges_train.csv", index=False)

class MLP2ImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):

        img_csv = self.img_labels.iloc[idx, 1:].values
        image = img_csv.reshape(28, 28)

        label = self.img_labels.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    

training_data_edge = MLP2ImageDataset(
    annotations_file = "edges_train.csv",
    transform=ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

test_data_edge = MLP2ImageDataset(
    annotations_file = "edges_test.csv",
    transform=ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

train_dataset_edge, val_dataset_edge = torch.utils.data.random_split(training_data_edge, [int(0.75*len(training_data_edge)), int(0.25*len(training_data_edge))])

from torch.utils.data import DataLoader
train_dataloader_edge = DataLoader(train_dataset_edge, batch_size=64, shuffle=True)
val_dataloader_edge = DataLoader(val_dataset_edge, batch_size=64, shuffle=True)
test_dataloader_edge = DataLoader(test_data_edge, batch_size=64, shuffle=True)


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

model = NeuralNetwork().to(device)
print(model)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0166,  0.0339,  0.0168,  ..., -0.0265, -0.0353,  0.0273],
        [-0.0074, -0.0002, -0.0197,  ...,  0.0271,  0.0028, -0.0253]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0114,  0.0135], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0251, -0.0136, -0.0076,  ..., -0.0144, -0.0318, -0.0333],
        [-0.0366,  0.0351,  0.0132,  ...,  0.0057, -0.0235,  0.0149]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

La

In [4]:
batch_size=64

from tqdm import tqdm

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    total = len(dataloader)

    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader),desc="train", total = total):
        # Compute prediction and loss
        X, y = X.to(device).float(), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



def test_loop(dataloader, model, loss_fn, model_path):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.load_state_dict(torch.load(model_path))

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


def val_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            
            y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    val_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Validation loss: {val_loss:>8f} \n")

    return correct

: 

In [ ]:
learning_rate_edge = 1e-3
batch_size_edge = 64
epochs_edge = 10

loss_fn_edge = nn.CrossEntropyLoss()
optimizer_edge = torch.optim.SGD(model.parameters(), lr=learning_rate_edge)

# model.to(device)
val_accuracy_highest_edge = 0
val_accuracy_curr_edge = 0

for t in range(epochs_edge):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader_edge, model, loss_fn_edge, optimizer_edge)
    val_accuracy_curr_edge = val_loop(test_dataloader_edge, model, loss_fn_edge)
    if val_accuracy_curr_edge > val_accuracy_highest_edge:
        val_accuracy_highest_edge = val_accuracy_curr_edge
        model_path = 'model_1_edge'
        print(f"Saving Model at epoch: {t+1}\n")
        torch.save(model.state_dict(), model_path)

    
print("Done!")

test_loop(test_dataloader_edge, model, loss_fn_edge, "model_1_edge")

Epoch 1
-------------------------------


train:   0%|          | 0/704 [00:00<?, ?it/s]

train: 100%|██████████| 704/704 [00:10<00:00, 64.09it/s]


Validation Error: 
 Accuracy: 76.0%, Avg Validation loss: 0.743723 

Saving Model at epoch: 1

Epoch 2
-------------------------------


train: 100%|██████████| 704/704 [00:10<00:00, 67.42it/s]


...

train: 100%|██████████| 704/704 [00:12<00:00, 57.56it/s]


Validation Error: 
 Accuracy: 80.4%, Avg Validation loss: 0.666988 

Epoch 9
-------------------------------


train: 100%|██████████| 704/704 [00:10<00:00, 65.70it/s]


Validation Error: 
 Accuracy: 81.3%, Avg Validation loss: 0.658830 

Saving Model at epoch: 9

Epoch 10
-------------------------------


train:  76%|███████▌  | 536/704 [00:09<00:04, 40.45it/s]